In [1]:
import pandas as pd

In [2]:
score = pd.read_csv("score.tsv", delimiter="\t")

In [67]:
import subprocess
from subprocess import Popen, PIPE, STDOUT
from pathlib import Path

def make_script(video_path, force=False):
    
    keypoint_path = Path(video_path).with_suffix(".json")
    if not force and keypoint_path.is_file():
        print("Keypoint Exist - ", keypoint_path)
        return

    cmd = [
        "python",
        "demo/body3d_two_stage_video_demo.py",
        "demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py",
        "https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth",

        "configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py",
        "https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth",
        "configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/videopose3d_h36m_243frames_fullconv_supervised_cpn_ft.py",
        "https://download.openmmlab.com/mmpose/body3d/videopose/videopose_h36m_243frames_fullconv_supervised_cpn_ft-88f5abbb_20210527.pth",
         
        "--video-path", video_path,
        "--keypoint-path", keypoint_path,
        "--out-video-root", "vis_results",
        "--num-instances", "1",
        "--rebase-keypoint-height",
    ]
    
    return cmd

In [68]:
processes = []

In [84]:

import os, re, mmcv

test_cnt = 10000
cmd_d = {}
basedir = '/mnt/c/Users/USER/Dropbox/머신러닝_수강생영상 (2)/'
for i, r in score.iterrows():
    title = r['title'].strip()
    filename = r['file name']
    score1 = r['Skill']
    score2 = r['Expression']

    title = re.sub(r"[\*\?]", "", title)
    filename = re.sub(r"[\*\?]", "", filename)

    found = False
    dir1 = Path(basedir) / title
    for file in os.listdir(dir1):
        if file.startswith(filename) and not file.endswith("json"):
            filename = file
            found = True

    if not found:
        print(i, "Video Not Found", dir1 / filename)
        continue

    x_path = dir1 / filename
    
    found = False
    dir2 = dir1 / "강사님 영상"
    for file in os.listdir(dir2):
        if not file.startswith(".") and not file.endswith("json"):
            y_path = dir2 / file
            found = True
    
    if not found:
        print(i, "Teacher Video Not Found", dir2)
        continue
    vid_x = mmcv.VideoReader(str(x_path))
    vid_y = mmcv.VideoReader(str(y_path))
    if abs(vid_x.fps - vid_y.fps) > 1:
        print(i, "FPS Problem", vid_x.fps, vid_y.fps)
        continue

    cmd = make_script(x_path, force=True)
    if cmd is not None:
        cmd_d[x_path] = cmd
        
        
    cmd = make_script(y_path, force=True)
    if cmd is not None:
        cmd_d[y_path] = cmd
    test_cnt -= 1
    if test_cnt == 0:
        break

1 FPS Problem 24.0 30.0
35 FPS Problem 24.0 30.0
45 FPS Problem 28.267379679144383 30.0
62 Video Not Found /mnt/c/Users/USER/Dropbox/머신러닝_수강생영상 (2)/Candy (Original)/0075_BAEKHYUN_Candy (Original)
66 FPS Problem 25.0 30.0
72 FPS Problem 24.0 30.0
75 FPS Problem 24.0 30.0
76 Teacher Video Not Found /mnt/c/Users/USER/Dropbox/머신러닝_수강생영상 (2)/Dancing Till We Drop/강사님 영상
80 Teacher Video Not Found /mnt/c/Users/USER/Dropbox/머신러닝_수강생영상 (2)/Deja Vu (Original) (Pt.1)/강사님 영상
81 Teacher Video Not Found /mnt/c/Users/USER/Dropbox/머신러닝_수강생영상 (2)/Deja Vu (Original) (Pt.2)/강사님 영상
94 FPS Problem 24.0 30.0
102 FPS Problem 14.251493549597392 29.97002997002997
115 FPS Problem 24.003908370426664 29.97002997002997
122 FPS Problem 24.0 29.97002997002997
132 FPS Problem 19.98001998001998 29.97002997002997
133 FPS Problem 24.0 29.97002997002997
153 FPS Problem 24.942263279445726 29.97002997002997
162 FPS Problem 24.0 29.97002997002997
167 FPS Problem 24.0 29.97002997002997
168 FPS Problem 24.0 29.97002997002997


In [85]:
keys = list(cmd_d.keys())
len(keys)

1677

In [86]:
import time

current_running = []
max_run = 2
while(len(keys) > 0):
    to_remove = []
    for p in current_running:
        if p.poll() is not None:
            to_remove.append(p)
    print(len(to_remove))
    if len(to_remove):
        for p in to_remove:
            current_running.remove(p)
    else:
        time.sleep(5)
    while(len(current_running) < max_run and len(keys) > 0):
        k = keys.pop()
        p = Popen(cmd_d[k])
        current_running.append(p)
for p in current_running:
    p.wait()

0
0
Keypoint Exist
Stage 1: 2D pose detection.
Initializing model...
Keypoint Exist
Stage 1: 2D pose detection.
Initializing model...
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
0
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
Running 2D pose detection inference...
[                                                  ] 0/1497, elapsed: 0s, ETA:Running 2D pose detection inference...
[                                ] 4/1497, 1.9 task/s, elapsed: 2s, ETA:   769s0
[                          

Traceback (most recent call last):
  File "/home/syd/workspace/mmpose/demo/body3d_two_stage_video_demo.py", line 591, in <module>
    main()
  File "/home/syd/workspace/mmpose/demo/body3d_two_stage_video_demo.py", line 398, in main
    for frame_id, cur_frame in enumerate(mmcv.track_iter_progress(video)):
  File "/home/syd/anaconda3/lib/python3.9/site-packages/mmcv/utils/progressbar.py", line 200, in track_iter_progress
    task_num = len(tasks)
ValueError: __len__() should return >= 0


[>                            ] 296/4426, 5.7 task/s, elapsed: 52s, ETA:   731s1
0
[>>                           ] 322/4426, 5.7 task/s, elapsed: 57s, ETA:   724sKeypoint Exist
Stage 1: 2D pose detection.
Initializing model...
[>>                           ] 324/4426, 5.7 task/s, elapsed: 57s, ETA:   723sload checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
[>>                           ] 325/4426, 5.7 task/s, elapsed: 57s, ETA:   723s0
[>>                           ] 337/4426, 5.7 task/s, elapsed: 59s, ETA:   720sload checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
[>>                           ] 341/4426, 5.7 task/s, elapsed: 60s, ETA:   720sRunning 2D pose detection inference...
[>>                           ] 353/4426, 5.7 task/s, elapsed: 62s, ETA:   720s0
[                               ] 2

Traceback (most recent call last):
  File "/home/syd/workspace/mmpose/demo/body3d_two_stage_video_demo.py", line 591, in <module>
    main()
  File "/home/syd/workspace/mmpose/demo/body3d_two_stage_video_demo.py", line 398, in main
    for frame_id, cur_frame in enumerate(mmcv.track_iter_progress(video)):
  File "/home/syd/anaconda3/lib/python3.9/site-packages/mmcv/utils/progressbar.py", line 200, in track_iter_progress
    task_num = len(tasks)
ValueError: __len__() should return >= 0


[>>>>>>>>>>>>>>>>>>>>       ] 1230/1587, 4.1 task/s, elapsed: 302s, ETA:    88s1
0
[>>>>>>>>>>>>>>>>>>>>>      ] 1253/1587, 4.1 task/s, elapsed: 306s, ETA:    82sKeypoint Exist
Stage 1: 2D pose detection.
Initializing model...
[>>>>>>>>>>>>>>>>>>>>>      ] 1254/1587, 4.1 task/s, elapsed: 306s, ETA:    81sload checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
0
[>>>>>>>>>>>>>>>>>>>>>      ] 1266/1587, 4.1 task/s, elapsed: 309s, ETA:    78sload checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
[>>>>>>>>>>>>>>>>>>>>>      ] 1269/1587, 4.1 task/s, elapsed: 310s, ETA:    78sRunning 2D pose detection inference...
[>>>>>>>>>>>>>>>>>>>>>      ] 1278/1587, 4.1 task/s, elapsed: 312s, ETA:    75s0
[                               ] 30/1487, 4.5 task/s, elapsed: 7s, ETA:   326s0
[>                             ] 

Traceback (most recent call last):
  File "/home/syd/workspace/mmpose/demo/body3d_two_stage_video_demo.py", line 591, in <module>
    main()
  File "/home/syd/workspace/mmpose/demo/body3d_two_stage_video_demo.py", line 398, in main
    for frame_id, cur_frame in enumerate(mmcv.track_iter_progress(video)):
  File "/home/syd/anaconda3/lib/python3.9/site-packages/mmcv/utils/progressbar.py", line 200, in track_iter_progress
    task_num = len(tasks)
ValueError: __len__() should return >= 0


[>>>>>>>>>>>>>>>>           ] 1312/2193, 5.5 task/s, elapsed: 237s, ETA:   159s1
0
[>>>>>>>>>>>>>>>>           ] 1337/2193, 5.5 task/s, elapsed: 242s, ETA:   155s0
[>>>>>>>>>>>>>>>>           ] 1338/2193, 5.5 task/s, elapsed: 242s, ETA:   155sKeypoint Exist
Stage 1: 2D pose detection.
Initializing model...
[>>>>>>>>>>>>>>>>           ] 1339/2193, 5.5 task/s, elapsed: 242s, ETA:   154sload checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
[>>>>>>>>>>>>>>>>           ] 1352/2193, 5.5 task/s, elapsed: 245s, ETA:   152sload checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
[>>>>>>>>>>>>>>>>           ] 1356/2193, 5.5 task/s, elapsed: 245s, ETA:   152sRunning 2D pose detection inference...
[>>>>>>>>>>>>>>>>           ] 1362/2193, 5.5 task/s, elapsed: 247s, ETA:   151s0
[                                ] 

In [80]:
for p in current_running:
    p.kill()

In [135]:
for p in current_running:
    p.wait()

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1176/1176, 5.4 task/s, elapsed: 217s, ETA:     0s
Stage 2: 2D-to-3D pose lifting.
Initializing model...
load checkpoint from http path: https://download.openmmlab.com/mmpose/body3d/videopose/videopose_h36m_243frames_fullconv_supervised_cpn_ft-88f5abbb_20210527.pth
Running 2D-to-3D pose lifting inference...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1176/1176, 12.2 task/s, elapsed: 96s, ETA:     0s
